In [1]:
playlist_input = "https://www.youtube.com/playlist?list=PLmbUlSQHQlvhEqMXztLb2ZnLDvg20J2nD"

In [2]:
import re

playlist_id = None
# Regex to extract playlist ID from various YouTube URL formats
playlist_url_regex = r'(?:youtube\.com\/(?:[^\/]+\/.+?\/|(?:v|e(?:mbed)?)\/|.*[?&]list=)|youtu\.be\/)([^"&?\/ ]{11,})'

match = re.search(playlist_url_regex, playlist_input)
if match:
    playlist_id = match.group(1)

# If it's not a URL or the regex failed, assume the input is directly the playlist ID
if not playlist_id:
    playlist_id = playlist_input

print(f"Extracted Playlist ID: {playlist_id}")

Extracted Playlist ID: PLmbUlSQHQlvhEqMXztLb2ZnLDvg20J2nD


In [3]:
import sys
!{sys.executable} -m pip install --upgrade google-api-python-client
print("Installed google-api-python-client.")

Installed google-api-python-client.


In [4]:
from googleapiclient.discovery import build
import os
from dotenv import load_dotenv
load_dotenv()

api_key = os.getenv("YOUTUBE_API_KEY")
# Placeholder for your API key. Replace with your actual key.
YOUTUBE_API_KEY = api_key

youtube = build("youtube", "v3", developerKey=YOUTUBE_API_KEY)
print("YouTube API service initialized.")

YouTube API service initialized.


In [9]:
max_videos = 20
print(f"Max videos to process set to: {max_videos}")

Max videos to process set to: 20


In [10]:
video_ids = []
next_page_token = None

while True:
    request = youtube.playlistItems().list(
        part="snippet",
        playlistId=playlist_id,
        maxResults=50, # Max results per page
        pageToken=next_page_token
    )
    response = request.execute()

    for item in response['items']:
        video_id = item['snippet']['resourceId']['videoId']
        video_ids.append(video_id)
        if len(video_ids) >= max_videos:
            break

    next_page_token = response.get('nextPageToken')
    if not next_page_token or len(video_ids) >= max_videos:
        break

# Truncate video_ids to ensure it does not exceed max_videos
video_ids = video_ids[:max_videos]

print(f"Found {len(video_ids)} video IDs.")
print("Video IDs:", video_ids)

Found 20 video IDs.
Video IDs: ['k5juTqk-OoA', 'VQo1Eut6jhA', 'hvHNbeTsneY', 'OgYJmEB7lQk', 'Zs7qVzVu8sA', '4raib3FSIdE', 'EHXRd4oNEqM', 'KA46LrBO7TM', '47KrW28W2YU', 'HEYfSKI-kWw', 'rK90XkzEvd0', 'dpwfv92XUIM', 'pbJIaUr-Sfg', 'c5C9wn6Kd-E', '1sS7XPc4PnQ', '14uO75YlYkA', 'k27pd5LLxEs', 'y3_4F0_PM_0', 'vaMPbvvFLXI', '5A2J7jSyTAo']


**Reasoning**:
The next step is to create a directory named `captions` to store the downloaded caption files.



In [11]:
import os

captions_dir = "captions"
os.makedirs(captions_dir, exist_ok=True)
print(f"Directory '{captions_dir}' created or already exists.")

Directory 'captions' created or already exists.


In [ ]:
import yt_dlp
import os

for video_id in video_ids:
    video_url = f"https://www.youtube.com/watch?v={video_id}"
    ydl_opts = {
        'writesubtitles': True,
        'writeautomaticsub': True,
        'subtitleslangs': ['pt'],  # Try Portuguese first, then English
        'subtitlesformat': 'srt',
        'skip_download': True,
        'outtmpl': os.path.join(captions_dir, '%(title)s-%(id)s.%(ext)s'),
        'quiet': False,  # Set to True to suppress output
        'no_warnings': False  # Show warnings to debug
    }

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(video_url, download=False)
            
            # Check available subtitles
            if 'subtitles' in info or 'automatic_captions' in info:
                available_subs = list(info.get('subtitles', {}).keys()) + list(info.get('automatic_captions', {}).keys())
                print(f"Video {video_id} - Available subtitles: {available_subs}")
            
            # Now try to download
            ydl.download([video_url])
            print(f"✓ Successfully downloaded captions for: {video_id}")
            
    except Exception as e:
        print(f"✗ Could not download captions for {video_id}. Error: {e}")

print(f"\nCaption download completed for {len(video_ids)} videos.")

[youtube] Extracting URL: https://www.youtube.com/watch?v=k5juTqk-OoA
[youtube] k5juTqk-OoA: Downloading webpage


[youtube] k5juTqk-OoA: Downloading android sdkless player API JSON
[youtube] k5juTqk-OoA: Downloading web safari player API JSON


[youtube] k5juTqk-OoA: Downloading m3u8 information


[info] k5juTqk-OoA: Downloading subtitles: pt, en


Video k5juTqk-OoA - Available subtitles: ['live_chat', 'ab', 'aa', 'af', 'ak', 'sq', 'am', 'ar', 'hy', 'as', 'ay', 'az', 'bn', 'ba', 'eu', 'be', 'bho', 'bs', 'br', 'bg', 'my', 'ca', 'ceb', 'zh-Hans', 'zh-Hant', 'co', 'hr', 'cs', 'da', 'dv', 'nl', 'dz', 'en', 'eo', 'et', 'ee', 'fo', 'fj', 'fil', 'fi', 'fr', 'gaa', 'gl', 'lg', 'ka', 'de', 'el', 'gn', 'gu', 'ht', 'ha', 'haw', 'iw', 'hi', 'hmn', 'hu', 'is', 'ig', 'id', 'iu', 'ga', 'it', 'ja', 'jv', 'kl', 'kn', 'kk', 'kha', 'km', 'rw', 'ko', 'kri', 'ku', 'ky', 'lo', 'la', 'lv', 'ln', 'lt', 'lua', 'luo', 'lb', 'mk', 'mg', 'ms', 'ml', 'mt', 'gv', 'mi', 'mr', 'mn', 'mfe', 'ne', 'new', 'nso', 'no', 'ny', 'oc', 'or', 'om', 'os', 'pam', 'ps', 'fa', 'pl', 'pt-orig', 'pt', 'pt-PT', 'pa', 'qu', 'ro', 'rn', 'ru', 'sm', 'sg', 'sa', 'gd', 'sr', 'crs', 'sn', 'sd', 'si', 'sk', 'sl', 'so', 'st', 'es', 'su', 'sw', 'ss', 'sv', 'tg', 'ta', 'tt', 'te', 'th', 'bo', 'ti', 'to', 'ts', 'tn', 'tum', 'tr', 'tk', 'uk', 'ur', 'ug', 'uz', 've', 'vi', 'war', 'cy', 'fy'

...

[info] 5A2J7jSyTAo: Downloading subtitles: pt, en


[info] 5A2J7jSyTAo: Downloading 1 format(s): 96
[info] Writing video subtitles to: captions\BOLSONARO PRESO, E AGORA？ ｜ ANÁLISES RENAIS ｜ 24⧸11⧸2025-5A2J7jSyTAo.pt.srt


[download] Destination: captions\BOLSONARO PRESO, E AGORA？ ｜ ANÁLISES RENAIS ｜ 24⧸11⧸2025-5A2J7jSyTAo.pt.srt
[download] 100% of  206.03KiB in 00:00:00 at 636.10KiB/s
[info] Writing video subtitles to: captions\BOLSONARO PRESO, E AGORA？ ｜ ANÁLISES RENAIS ｜ 24⧸11⧸2025-5A2J7jSyTAo.en.srt


ERROR: Unable to download video subtitles for 'en': HTTP Error 429: Too Many Requests


✗ Could not download captions for 5A2J7jSyTAo. Error: ERROR: Unable to download video subtitles for 'en': HTTP Error 429: Too Many Requests

Caption download completed for 20 videos.
